In [1]:
import os
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from piq import ssim


In [2]:
def psnr(input, target, data_range=1.0):
    mse = torch.mean((input - target) ** 2)
    if mse == 0:
        return float('inf')
    return 10 * torch.log10((data_range ** 2) / mse)


In [3]:
image_folder = '/home/piai/문서/miryeong/ImagePreprocessor/평가지표/image_folder'
target_folder = '/home/piai/문서/miryeong/ImagePreprocessor/평가지표/target_folder' 

In [10]:
image_files = [os.path.join(image_folder, file) 
               for file in os.listdir(image_folder) 
               if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

psnr_list = []
ssim_list = []

preprocess = transforms.Compose([transforms.ToTensor()])

for image_file in image_files:
    print('image:', image_file)
    image = Image.open(image_file).convert("RGB")
    image = preprocess(image)
    # print('image shape:', image.shape)
    
    transform = transforms.Compose([transforms.Resize((image.shape[1], image.shape[2]))])
    
    target_file = os.path.join(target_folder, os.path.basename(image_file))
    if os.path.exists(target_file):
        target_image = Image.open(target_file).convert("RGB")
        target_image = transform(target_image)
        
        target_image = preprocess(target_image)
        image = image.unsqueeze(0)  # 차원 추가
        target_image = target_image.unsqueeze(0)  # 차원 추가
        # print('target image shape:', target_image.shape)
        
        psnr_value = psnr(image, target_image, data_range=1.0)
        print('psnr:', psnr_value)
        psnr_list.append(psnr_value)
        
        ssim_value = ssim(image, target_image, data_range=1.0)
        print('ssim:', ssim_value)
        ssim_list.append(ssim_value)
        
        print("{} - PSNR : {} SSIM: {}".format(image_file.split('/')[-1],psnr_value,ssim_value))
        
        

psnr_mean = torch.mean(torch.stack(psnr_list))
ssim_mean = torch.mean(torch.stack(ssim_list))
print("Average - PSNR : {} SSIM: {}".format(psnr_mean.item(),ssim_mean.item()))

image: /home/piai/문서/miryeong/ImagePreprocessor/평가지표/image_folder/saved_image1.png
psnr: tensor(9.9312)
ssim: tensor(0.4726)
saved_image1.png - PSNR : 9.931161880493164 SSIM: 0.47263702750205994
image: /home/piai/문서/miryeong/ImagePreprocessor/평가지표/image_folder/saved_image2.png
psnr: tensor(16.8851)
ssim: tensor(0.8017)
saved_image2.png - PSNR : 16.88507652282715 SSIM: 0.801703929901123
image: /home/piai/문서/miryeong/ImagePreprocessor/평가지표/image_folder/saved_image3.png
psnr: tensor(13.6991)
ssim: tensor(0.7680)
saved_image3.png - PSNR : 13.699102401733398 SSIM: 0.7680030465126038
image: /home/piai/문서/miryeong/ImagePreprocessor/평가지표/image_folder/saved_image4.png
psnr: tensor(10.6904)
ssim: tensor(0.6605)
saved_image4.png - PSNR : 10.690431594848633 SSIM: 0.6604847311973572
Average - PSNR : 12.801443099975586 SSIM: 0.675707221031189
